In [339]:
# 预测结果
import pandas as pd
import numpy as np
from collections import Counter

from sklearn.multioutput import MultiOutputClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
import jieba

save = False
# 初始化参数
df = pd.read_excel('demo.xlsx',sheet_name='demo',usecols=[0,1,2,3])

df['code'] = df['code'].apply(lambda x: x.split(','))
all_labels = [item for sublist in df['code'] for item in sublist]
label_counts = Counter(all_labels)
rare_labels = {label for label, count in label_counts.items() if count == 1}
# 稀有类别对验证没有任何帮助，直接去除
df['code'] = df['code'].apply(lambda x: np.unique( [item for item in x if item not in rare_labels]))
# rare_labels,np.unique(df['code'].values)
df = df[df['code'].apply(lambda x: len(x)>0)]
# 构建特征, 使用l2进行归一化，smooth_idf 方式分母为0
tf = TfidfVectorizer(norm="l2",smooth_idf=True)
df['feature'] = df[['series', 'type', 'name']].astype(str).apply(' '.join, axis=1)
X = tf.fit_transform(df['feature'])
mlb = MultiLabelBinarizer()

y = mlb.fit_transform(df['code'])

# 方差过滤
from sklearn.feature_selection import VarianceThreshold

X_var = VarianceThreshold().fit_transform(X)


# 方差太小，说明特征无效
X_pd = pd.DataFrame(X.toarray())
X_pd.var().mean(),X_pd.var().median(),X_pd.var().min(),X_pd.var().max()

(np.float64(0.000591226831008344),
 np.float64(0.00024186554835175925),
 np.float64(3.689923085161966e-05),
 np.float64(0.02073341825216165))

In [378]:
# 尝试直接count
import jieba
import joblib
from sklearn.feature_extraction.text import  CountVectorizer
# 尝试清理噪音 ,"",""
stop=["-","."," ","【", "】", "+","（","）","(",")","*",",","/","=","|",";","以上","如果","邮费","\t","单条装",
      "一条","两条装","其他","，","一个","三条","超级","开盖","公对公","不伤机","不含线","2件","4件",
      "1条装","2条装","1条","2条","3条","条","两条","单条","没有","光速","加长","充电器","显示","是","笔记本电脑","优惠装","搭配装","优惠","链接","同长","默认","爆款","秒","备注","品牌"
      ,"发","冲头","两","电竞版","请","充电数据线","安卓快充电线","快充电线","充电线","闪充线","闪充数据线","看后面","后面","普通","单个头","快充头","快充线","正品","专用头","专用"
      ,"2个","1个","3个","2根","3根","6根","单个","单头","闪充头","闪充","快充","秒充","线长","通用","极速","系列","适配器","配","准","系列","单","装","充电","长",'发2根','发3根'
      ,"加长版","正常","总链接","电线","双引擎","单线","头","秒冲","to","cc","6","6t","5","5t","3","3t","9","1619","a1398","a1424",'a1181','a1278','a1286','a1343','a1369','a1370'
 ,'a14','a1425','a1465', 'a1502', 'a1534', 'a1706', 'a1707', 'a1708', 'a1932','e'
 ,'a1990' ,'a21' ,'a2141', 'a2363', 'a2518','po']

synonym = {"iapd":"ipad","编制":"编织","华为":"华","平果":"苹果","viov":"vo","vovi":"vo","v":"vo","v":"vo","vooc":"vo","vi":"vo"
            ,"120":"120w","150":"150w","40":"40w","45":"45w","66":"66w","88":"88w","88":"88w","67":"67w","80":"80w","90":"90w","100":"100w","33":"33w"
          ,"oppo":"op","绿口":"绿口线","紫口":"紫口线","白口":"白口线","黄口":"白口线","黄":"白口线"
          ,"一嘉":"一加","转":"to","加":"一加","mi":"小米","米":"小米"
          ,"lightning":"扁口","线":"数据线","mirco":"micro","reno9":"r9"
          ,"tpc口":"c口","看后面":"后面","米6a线":"6a","器":"头","套装":"头"
          ,"米6a":"6a","高攻":"g","高功":"g","高":"g","sam星":"三星","1":"1米","2":"2米","1.5":"1.5米","1..5米":"1.5米"
          ,"华为":"华","typec":"c","type-c":"c","tpc":"c","荣耀":"荣"}

# jieba.initialize()
to = jieba.Tokenizer()

to.load_userdict("userdict.txt")
to.del_word("po65")
def chinese_tokenizer(context):
    # 使用正则表达式去除自定义停止词
    context = context.lower()
    words = to.lcut(context,cut_all=False,HMM=False,use_paddle=True)
    # 同义词替换
    words = map(lambda x: synonym[x] if x in synonym else x,words)
    # 去除停用词
    words =  [*filter(lambda x: x not in stop,words)]
    if any(word == "a" for word in words):
        print(context)
    return words


vectorizer = CountVectorizer(
    analyzer=chinese_tokenizer,
    token_pattern=None,
    lowercase=True,
    binary=True
)
# print(df['feature'].head(5))
X = vectorizer.fit_transform(df['feature'])
y = mlb.fit_transform(df['code'])
if save:
    joblib.dump(vectorizer,"vectorizer.bin")
print(vectorizer.get_feature_names_out())

# 方差过滤
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA


X_var = X

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache


Loading model cost 0.508 seconds.
Prefix dict has been built successfully.


华 数据线,充电头 华88w双口头g【a+c】【套装】88w快充头+2米华6a线
华 数据线,充电头 华88w双口头g【a+c】【套装】88w快充头+1米华6a线
华 数据线,充电头 华88w双口头g【a+c】【套装】88w快充头+1.5米华6a线
华 数据线,充电头 华100w双口头【a+c】【100w快充头+2米线】
华 数据线,充电头 华100w双口头【a+c】【100w快充头+1米线+2米线】
华 数据线,充电头 华100w双口头【a+c】【100w快充头+1米线+1.5米线】
华 数据线,充电头 华100w双口头【a+c】【100w快充头+1米线】
华 数据线,充电头 华100w双口头【a+c】【100w快充头+1.5米线+2米线】
华 数据线,充电头 华100w双口头【a+c】【100w快充头+1.5米线】
华 充电头 华100w双口头【a+c】【100w快充头单头】
华 充电头 华100w双口头g【a+c】【单头】100w快充头
华 充电头 华100w双口头【a+c】
华 数据线,充电头 华100w双口头g【a+c】【套装】100w快充头+1.5米华6a线
华 数据线,充电头 华100w双口头【a+c】+1.5米线
华 数据线,充电头 华100w双口头【a+c】+1米线
华 数据线,充电头 华100w双口头g【a+c】【套装】100w快充头+1米华6a线
华 数据线,充电头 华100w双口头g【a+c】【套装】100w快充头+2米华6a线
华 充电头 华双口头【a+c】【66w双口快充头单头】
华 充电头 华双口头【a+c】g【66w双口快充头单头】
华 充电头 华66w双口头【a+c】g配双c线
华 充电头 华66w双口头【a+c】g配双c线【66w双口快充头】
华 数据线,充电头 华66w双口头【a+c】g配双c线【66w双口快充头+1.5米线双c线】
华 数据线,充电头 华66w双口头【a+c】g配双c线【66w双口头+1.5米线+2米双c线】
华 数据线,充电头 华66w双口头【a+c】配双c线【66w双口头+1.5米线+2米双c线】
华 数据线,充电头 华66w双口头【a+c】g配双c线【66w双口快充头+1米线双c线】
华 数据线,充电头 华66w双口头【a+c】配双c线【66w双口快充头+1米线双c线】
华 数据线,充电头 华66w双口头

In [ ]:
# 预测系列
import pandas as pd
import numpy as np
from collections import Counter

from sklearn.multioutput import MultiOutputClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelEncoder
import joblib
from sklearn.model_selection import cross_val_score,StratifiedKFold
# 初始化参数
series_df = pd.read_excel('demo.xlsx',sheet_name='demo',usecols=[0,2])

all_labels = [item for item in series_df['series']]

X = vectorizer.fit_transform(series_df['name'])
print(X.shape)
encoder = LabelEncoder()
y = encoder.fit_transform(series_df['series'])
series_model = DecisionTreeClassifier(criterion="log_loss",max_depth=10,random_state=42)
series_model.fit(X,y)
skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=42)
if save:
    joblib.dump(encoder, "series_encoder.bin")
    joblib.dump(series_model, "series_model.bin")

cross_val_score(series_model,X,y,cv=skf,scoring="accuracy").mean()


(4493, 199)


np.float64(0.9534828354197066)

In [ ]:
# 预测类型
import pandas as pd
import numpy as np
from collections import Counter

from sklearn.multioutput import MultiOutputClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.model_selection import StratifiedKFold
import joblib
# 初始化参数
type_df = pd.read_excel('demo.xlsx',sheet_name='demo',usecols=[0,3])

type_df['type'] = type_df['type'].apply(lambda x: x.split(','))

mlb = MultiLabelBinarizer()

X = vectorizer.fit_transform(type_df['name'])
y = mlb.fit_transform(type_df['type'])
type_model = MultiOutputClassifier(DecisionTreeClassifier(criterion="log_loss",max_depth=10,random_state=42))
type_model.fit(X,y)
if save:
    joblib.dump(mlb, "type_encoder.bin")
    joblib.dump(type_model, "type_model.bin")
sfk = MultilabelStratifiedKFold(n_splits=5,shuffle=True,random_state=42)
cross_val_score(type_model,X,y,cv=sfk,scoring="accuracy").mean()

np.float64(0.8795675052623283)

In [377]:
# 尝试嵌入法，下过变差了，threshold=0.0005 差不多，不能过大，缩减到96个特征  1/219 ,每个特征的重要性大概 0.005
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectFromModel

from sklearn.tree import DecisionTreeClassifier
print(X_var.shape)
embedded_mode = SelectFromModel(DecisionTreeClassifier(criterion="entropy",random_state=42), threshold=0.0005)
embedded_mode.fit(X_var,y)
X_embedded = embedded_mode.transform(X_var)
print(X_embedded.shape,vectorizer.get_feature_names_out()[embedded_mode.get_support()])

# X_embedded = VarianceThreshold().fit_transform(X_embedded)


# 方差虽然变大了，但是特征少了
# X_pd = pd.DataFrame(X_embedded.toarray())
# X_pd.var().mean(),X_pd.var().median(),X_pd.var().min(),X_pd.var().max(),X_pd

(4493, 163)
(4493, 77) ['1.5米' '100w' '10w' '120w' '12w' '150w' '1米' '200w' '240w' '25w' '29w'
 '2米' '30w' '33w' '40w' '44w' '45w' '55w' '5a' '65w' '66w' '67w' '6a'
 '80w' '88w' '90w' 'a' 'c' 'ctoc口' 'c口' 'd' 'g' 'l型接口' 'max' 'op' 'pd'
 'pro' 'r9' 't型接口' 'usb' 'vo' '一加' '三星' '主推' '充电头' '全兼容' '努比亚' '华' '原ic头'
 '原ic头g' '双' '双口' '双口头' '大壳' '大壳头g' '安卓' '小布丁' '小米' '弯头' '数据线' '梯形' '白'
 '白口线' '白色' '笔记本' '紫口线' '红线' '绿口线' '编织' '胶囊' '苹果' '荣' '荣max' '金标' '金标头'
 '金标线' '黄口线']


In [396]:
# 交叉验证
from sklearn.multioutput import MultiOutputClassifier
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.model_selection import cross_val_score,KFold
from sklearn.tree import DecisionTreeClassifier,ExtraTreeClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC




# 假设你有 X (特征), y (标签)
skf = MultilabelStratifiedKFold(n_splits=5,shuffle=True,random_state=42)

md = MultiOutputClassifier(DecisionTreeClassifier(criterion="log_loss",max_depth=12,random_state=42,splitter="best",min_samples_split=2,min_samples_leaf=1))
if save:
    joblib.dump(embedded_mode, "embedded_model.bin")    
    # X_embedded  X_var
    print(X_embedded.shape)
    md.fit(X_embedded,y)
    joblib.dump(md, "model.bin")
    joblib.dump(mlb, "encoder.bin")
# 使用 cross_val_score 并指定 scoring 参数 X_embedded X_var
cross_val_score(md,X_embedded,y,cv=skf,scoring="accuracy").mean()

np.float64(0.8780529242292022)

In [337]:
# 使用模型
from sklearn.multioutput import MultiOutputClassifier
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.model_selection import cross_val_score,KFold
from sklearn.tree import DecisionTreeClassifier,ExtraTreeClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
import joblib
# 假设你有 X (特征), y (标签)
skf = MultilabelStratifiedKFold(n_splits=5)

md = MultiOutputClassifier(DecisionTreeClassifier(criterion="log_loss",max_depth=10,splitter="best"))

demo_df = pd.read_excel("D:/work/采购/现实仓库/demo_.xlsx",usecols=["商品简称"])
demo_df["商品简称"] = demo_df["商品简称"].str.replace("\t","")
# 使用商品简称预测 系列
series_model = joblib.load("series_model.bin")
X_input = vectorizer.transform(demo_df["商品简称"])
n_pred = series_model.predict(X_input)
encoder = joblib.load("series_encoder.bin")
predicted_codes = encoder.inverse_transform(n_pred)
demo_df['series'] = predicted_codes

# 预测 类型
type_model = joblib.load("type_model.bin")
X_input = vectorizer.transform(demo_df["商品简称"])
n_pred = type_model.predict(X_input)
encoder = joblib.load("type_encoder.bin")
predicted_codes = encoder.inverse_transform(n_pred)
re = []
for code in predicted_codes:
    s = ",".join(code)
    re.append(s)
demo_df['type'] = re
demo_df['feature'] = demo_df[['series', 'type', '商品简称']].astype(str).apply(' '.join, axis=1)
md.fit(X_var,y)

X_input = vectorizer.transform(demo_df["feature"].values.ravel())
print(X_input.shape)
embedded_model = joblib.load("embedded_model.bin")

X_embedded_use = X_input[:,embedded_mode.get_support()]
print(X_embedded_use.shape)
model = joblib.load("model.bin")
n_pred = model.predict(X_embedded_use)
encoder = joblib.load("encoder.bin")
predicted_codes = encoder.inverse_transform(n_pred)
re = []
for code in predicted_codes:
    s = ",".join(code)
    re.append(s)
demo_df['预测结果'] = re
demo_df.to_excel("D:/work/采购/现实仓库/demo_.xlsx")


(1, 199)
(1, 77)


In [208]:
# 尝试包装法
# 包装法 , 逻辑回归优先使用嵌入法，svm 优先使用包装法
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

rfc = DecisionTreeClassifier(criterion="entropy",random_state=42)
rfe = RFE(rfc, n_features_to_select=80)
X_rfe = rfe.fit_transform(X, y)

In [16]:
# 尝试结果
from sklearn.multioutput import MultiOutputClassifier
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.model_selection import cross_val_score,KFold
from sklearn.tree import DecisionTreeClassifier,ExtraTreeClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
# 假设你有 X (特征), y (标签)
skf = MultilabelStratifiedKFold(n_splits=5)

md = MultiOutputClassifier(RandomForestClassifier(n_estimators=10,criterion="entropy"))


md.fit(X_var,y)

X_input = vectorizer.transform(["华66双口头【66W快充头+1米线】"])
n_pred = md.predict(X_input)
predicted_codes = mlb.inverse_transform(n_pred)
predicted_codes


[(np.str_('华66w双口头【A+C】-25w'), np.str_('华6a线-1米'))]

In [ ]:
# 持久化
from sklearn.multioutput import MultiOutputClassifier
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.model_selection import cross_val_score,KFold
from sklearn.tree import DecisionTreeClassifier,ExtraTreeClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
import joblib
# 假设你有 X (特征), y (标签)
skf = MultilabelStratifiedKFold(n_splits=5)

md = MultiOutputClassifier(DecisionTreeClassifier(criterion="log_loss",max_depth=10))






(4493, 77)


['encoder.bin']

In [ ]:
import joblib
md = joblib.load("model.bin")
X_input = vectorizer.transform(["华 充电头 数据线 华双口66w 充电头+1米线"])
print(X_input.shape)
X_input = X_input[:,embedded_mode.get_support()]
print(X_input.shape)
n_pred = md.predict(X_input)
predicted_codes = mlb.inverse_transform(n_pred)
predicted_codes

(1, 242)
(1, 81)


[(np.str_('华66w双口头【A+C】-25w'), np.str_('华6a线-1米'))]